In [49]:
## Text Mining Project
# imports
import pandas as pd
import json
import re
import lyricsgenius

In [5]:
# Initialize genius (https://github.com/johnwmillr/LyricsGenius)
token = "jseKMRjvPp2qeRl5q9M2PpNUFtBJ-KeyTOpPHmOriyhSmSiFUKysDIceAbxWoBeT"
genius = lyricsgenius.Genius(token)

Searching for songs by Trailerpark...

Song 1: "Alles Für Ein Shirt"
Song 2: "Als gelesen markiert"
Song 3: "Aragorn"

Reached user-specified song limit (3).
Done. Found 3 songs.
[('Alles Für Ein Shirt', 'Trailerpark'), ('Als gelesen markiert', 'Trailerpark'), ('Aragorn', 'Trailerpark')]


In [61]:
# Download the lyrics
all_artists = ['187 Strassenbande', 'AK AUSSERKONTROLLE', 'Absolute Beginner', 'Alligatoah', 'Apache 207', 'Azad', 'Azet', 'Badmómzjay', 'Bausa', 'Beginner', 'Bonez MC', 'Bushido', 'Capital Bra', 'Casper', 'Celo & Abdi', 'Chakuza', 'Cro', 'Culcha Candela', 'CÉLINE', 'Dardan', 'Deichkind', 'Die Atzen', 'Die Fantastischen Vier', 'Eko Fresh', 'Eno', 'Farid Bang', 'Fero47', 'Fettes Brot', 'Fler', 'Freundeskreis', 'Genetikk', 'Gzuz', 'Haftbefehl', 'Jamule', 'Jan Delay', 'Joker Bra', 'Juju', 'K.I.Z', 'KC Rebell', 'Kay One', 'Kollegah', 'Kontra K', 'Kool Savas', 'Kraftklub', 'Loredana', 'Luciano', 'MERO', 'Marteria', 'Max Herre', 'MoTrip', 'Money Boy', 'Moses Pelham', 'Nana', 'Nimo', 'Olexesh', 'Peter Fox', 'Prinz Pi', 'RAF Camora', 'RIN', 'Rödelheim Hartreim Projekt', 'SSIO', 'SXTN', 'Sabrina Setlur', 'Samra', 'Samy Deluxe', 'Seeed', 'Shindy', 'Sido', 'Silla', 'Summer Cem', 'Tic Tac Toe', 'Trailerpark', 'Trettmann', 'Ufo361', 'Vega', 'Veysel', 'XATAR', 'Xavas', 'Yung Hurn', 'Zuna']
for artist_name in all_artists:
    artist = genius.search_artist(artist_name, sort="title")
    artist.save_lyrics()

TypeError: 'Song' object is not subscriptable

In [ ]:
# Store lyrics and relevant metadata in dataframe
all_songs = pd.DataFrame(columns = ['Title', 'Artist', 'Release', 'Lyrics'])
for artist_name in all_artists:
    with open(f'Lyrics_{artist_name}.json') as json_file:
        data = json.load(json_file)
        for song in data['songs']:
            title = song['title']
            release = song['release_date']
            lyrics_raw = song['lyrics']
            lyrics = re.sub("[\[].*?[\]]", "", lyrics_raw)
            new_song = {'Title': title, 'Artist': artist_name, 'Release': release, 'Lyrics': lyrics}
            all_songs = all_songs.append(new_song, ignore_index=True)